&emsp; 1 &ensp; CoLLiE 是什么

&emsp; &emsp; 1.1 &ensp; CoLLiE 的功能

&emsp; &emsp; 1.2 &ensp; CoLLiE 的安装

&emsp; &emsp; 1.3 &ensp; CoLLiE 的结构

## 1. CoLLiE 是什么

<!-- &emsp;; 多年以后，面对行刑队，奥雷里亚诺·布恩迪亚上校将会回想起父亲带他去见识冰块的那个遥远的下午。当时，马孔多是个二十户人家的村庄，一座座土房都盖在河岸上，河水清澈，沿着遍布石头的河床流去，河里的石头光滑、洁白，活象史前的巨蛋。 -->

&ensp; &ensp; 多年以后，面对AIGC，NLPer将会回想起OpenAI推出ChatGPT前的那个遥远的下午。当时，对于LLM的研究，只是OpenAI、Google、Meta几家科技寡头间的军备竞赛。然而，一切都随着那个时刻的降临而改变；随之而来的，是普罗大众对于LLM模型开源的呼声，以及对LLM高效微调的需求。

&ensp; &ensp; 模型的开源是学术研究的前提。在应用层面，OpenLMLab研发的[**MOSS**](https://github.com/OpenLMLab/MOSS)，作为国内首个类ChatGPT模型，为LLM的开源树立了良好的榜样。在模型层面，Meta的[**LLaMA**](https://arxiv.org/pdf/2302.13971.pdf)，作为一个参数量含盖 7B、13B、30B、65B 的 LLM开源集合，为LLM的深入研究和应用提供了基座。

&ensp; &ensp; 然而，开源并不意味着故事的结束；相反地，真正的问题才刚刚开始。如何仅以用户级的硬件配置（Consumer Hardware），即在有限的计算和存储资源，例如 **1张** 或 **8张 3090显卡** 上，实现对LLM，例如 **7B** 乃至 **65B LLaMA的全参数微调**，成为了接踵而至的问题。而这就是CoLLiE所解决的问题。

<!-- 虽然LLaMA是大羊驼，CoLLiE是牧羊犬，这个寓意是挺好的，但是如何表达就是一个比较头疼的问题，还得再想想 -->

### 1.1 CoLLiE 的功能

&ensp; &ensp; 训练角度：四种并行策略的整合：**数据并行**、**流水线并行**、**张量并行**、**ZeRO并行**

&ensp; &ensp; 微调角度：提出面向全参数高效微调的**InplaceSGD算法**，支持**LoRA**、**PrefixTuning**、**PromptTuning**等参数高效微调方案

&ensp; &ensp; 模型角度：兼容**FlashAttention**框架，极大提升Transformer运算效率

&ensp; &ensp; 使用角度：通过**Trainer**、**Config**等模块，允许用户轻松上手LLM微调

#### &emsp; 模型并行方案整合

&ensp; &ensp; 流水线并行、张量并行（在batchsize很小时相较于pp有优势）、ZeRO并行，本质都是将模型切分，分发到不同的设备上，不同之处在于，其参数切分和彼此通信的方式（打一个表格）

#### &emsp; 实现高效微调算法

&ensp; &ensp; 重点描述下InplaceSGD（但是流水线不支持，1F1B）

&ensp; &ensp; 图表描述下参数高效方法

#### &emsp; 结合高效计算方法

&ensp; &ensp; 图文结合描述一下

#### &emsp; 轻松定义训练过程

&ensp; &ensp; Trainer、Config大概干了啥

### 1.2 CoLLiE 的安装

&ensp; &ensp; CoLLiE通过git结合pip install安装，只需要一行指令如下。

```sh
pip install git+https://github.com/OpenLMLab/collie.git
```

&ensp; &ensp; CoLLiE在安装时需要保证保证环境已有的torch对应cuda版本和nvcc -V结果一致，例如笔者的nvcc是11.7版本，对应torch使用1.13.1+cu117

``` bash
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
```

&ensp; &ensp; 关于CoLLiE的依赖库，笔者简要列举了自己安装CoLLiE的环境中主要的包；其中较为重要的、在requirements.txt中出现的，使用粗体强调。其中，有三个库需要额外强调一下。apex、megatron、deepspeed（？？？）

| 名称 | 版本 |   | 名称 | 版本 |
|:----|:----|----|:----|:----|
| accelerate | 0.18.0 || **protobuf** | 3.20.1 |
| **apex** | 0.1 || requests | 2.28.2 |
| collie | 0.1.1 || **rich** | 11.2.0 |
| datasets | 2.4.0 || **sentencepiece** | 0.1.98 |
| **deepspeed** | 0.9.0 || setuptools | 66.0.0 |
| **einops** | 0.6.1 || tokenizers | 0.13.3 |
| FastNLP | 1.0.1 || **torch** | 1.13.1+cu117 |
| **fastapi** | 0.96.0 || **tqdm** | 4.65.0 |
| **flash-attn** | 1.0.4 || **transformers** | 4.28.1 |
| **megatron-core** | 0.1.0 || typing_extensions | 4.5.0 |
| numpy | 1.24.2 || wandb | 0.14.2 |
| pip | 23.0.1 || **websockets** | 11.0.3 |

&ensp; &ensp; 除此之外，CoLLiE提供了与torchrun和slurm的集成，以便用户在使用时可以轻松地在单个或多个节点上启动任务；详细内容会在后续tutorial中介绍。

### 1.3 CoLLiE 的结构

&ensp; &ensp; 笔者通过以下的表格简要描述了CoLLiE的文件结构，以及代码文件对应的功能，包括CoLLiE的实现代码，见collie/，以及辅助的演示代码，见examples/

| 路径 | 功能 |
|:----|:----|
|├─ **collie** |  |
|│&ensp;&ensp;&ensp;├─ **callbacks** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ callback.py | 定义CoLLiE的回调机制基类，明确了CoLLiE的回调时机 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ callback_manager.py | 定义CoLLiE回调类的管理模块，对应时机执行对应回调函数 |
|│&ensp;&ensp;&ensp;├─ config.py | 定义CoLLiE的配置类，例如seed、pp_size、tp_size、dp_size等 |
|│&ensp;&ensp;&ensp;├─ **controller** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ evaluator.py | 定义CoLLiE的测试模块Evaluator，方便用户快速评测模型性能 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ trainer.py | 定义CoLLiE的训练模块Trainer，方便用户快速实现分布式训练和测试 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ utils.py | 定义CoLLiE训练过程中回调函数的触发器TrainerEventTrigger |
|│&ensp;&ensp;&ensp;├─ **data** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ batch_sampler.py | 定义CoLLiE在最后个batch不满时的处理方式 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ dataloader.py | 定义CoLLiE的DataLoader，训练过程中每次迭代出若干条数据 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ dataset.py | 定义CoLLiE的Dataset，继承自torch.utils.data.Dataset |
|│&ensp;&ensp;&ensp;├─ **driver** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ **io** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ base.py | 定义CoLLiE的读写类IODriver |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ file.py | 定义CoLLiE的文件读写类，继承自IODriver |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;└─ petrel.py | ？？？ |
|│&ensp;&ensp;&ensp;├─ **log** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ handler.py | 定义CoLLiE的输出行为TqdmLoggingHandler、StdoutStreamHandler |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ highlighter.py | 定义CoLLiE的输出高亮行为ColorHighlighter |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ logger.py | 定义CoLLiE的日志记录模块Logger，封装了logging.Logger以及上述的模块 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ print.py | 定义CoLLiE的print函数，调用Logger模块，自定义CoLLiE的print方式 |
|│&ensp;&ensp;&ensp;├─ **metrics** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ accuracy.py | 定义CoLLiE的准确率，update需传入 {"pred": xxx, "target": xxx} |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ base.py | 定义CoLLiE的评测基类，gather收集各卡数据，update计算指标，get_metric统计输出 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ classify_f1_pre_rec_metric.py | 定义CoLLiE的F1值，update需传入 {"pred": xxx, "target": xxx} |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ decode.py | 定义CoLLiE的解码行为，update需传入 {"generated_ids": xxx}，get_metric直接接码生成 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ ppl.py | 定义CoLLiE的困惑度，update需传入 {"ppl": xxx} |
|│&ensp;&ensp;&ensp;├─ **models** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ base.py | 定义CoLLiE的语言模型基类CollieModelForCausalLM，提供from_pretrained等方法 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ **chatglm** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ model.py | 定义CoLLiE的ChatGLMForCausalLM，以及其内部模块 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ **llama** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ model.py | 定义CoLLiE的LlamaForCausalLM，以及其内部模块 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ **moss** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ model.py | 定义CoLLiE的MossForCausalLM，以及其内部模块 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;└─ utils.py | 定义CoLLiE在实现MossForCausalLM时，需要的辅助函数 |
|│&ensp;&ensp;&ensp;├─ module.py | 定义CoLLiE在实现上述CollieModelForCausalLM时，涉及到的参数并行模块 |
|│&ensp;&ensp;&ensp;├─ **optim** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ adan.py | 定义CoLLiE版本的Adan优化器，参考[Adaptive Nesterov Momentum Algorithm](https://arxiv.org/abs/2208.06677) |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ inplace_sgd.py | 定义CoLLiE提出的InplaceSGD优化器，参考xxx |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ lion.py | 定义CoLLiE版本的Lion优化器 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ sophiag.py | 定义CoLLiE版本的SophiaG优化器 |
|│&ensp;&ensp;&ensp;└─ **utils** |  |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ data_provider.py | 定义CoLLiE的异步数据提供器GradioProvider等 |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ dist_utils.py | 定义CoLLiE用于分布式以及环境初始化相关的辅助函数 |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ metric_wrapper.py | 定义CoLLiE封装metrics的模块MetricsWrapper |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ monitor.py | 定义CoLLiE的各种监控器，记录模型训练过程中的统计信息 |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ padder.py | 定义CoLLie的通用collate_fn构造器，默认左填充padding_left=True |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ peft_utils.py | 定义从peft到CoLLiE的适配函数patch_peft_model和patch_prompt_tuning |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ rich_progress.py | 定义CoLLiE中进度条打印的格式 |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;├─ seq_len_to_mask.py | 定义seq_len_to_mask函数，将seq_len数组转换为mask矩阵，不包含位置为0 |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;└─ utils.py | 定义CoLLiE中的其他辅助类或函数 |
|├─ **examples** |  |
|│&ensp;&ensp;&ensp;├─ **alpaca** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ alpaca.py | 定义Alpaca的数据集AlpacaDataset、训练和测试时的collate_fn |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ alpaca_data.json | 保存Alpaca的训练测试数据 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ alpaca_metric.py | 定义Alpaca的解码评测模块AlpacaDecodeMetric |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ base.py | 定义BaseAlpaca类，传入CollieConfig初始化，即可基于LLaMa-7B训练一个Alpaca |
|│&ensp;&ensp;&ensp;├─ finetune_llama.py | 基于collie.LlamaForCausalLM微调LLaMA-7B实现imdb分类，通过torchrun等方式启动 |
|│&ensp;&ensp;&ensp;├─ **finetune_moss** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ metric.py | 定义SFT评测正确率SFTAccMetric、SFT解码指标FTDecodeMetric |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ moss_002_sft.py | 定义SFT数据集格式，以及加载、预处理、补齐数据的函数 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ train.py | 基于collie.MossForCausalLM实现指令微调SFT，通过torchrun等方式启动 |
|│&ensp;&ensp;&ensp;├─ **generation** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ batch_generation.py | 基于collie.LlamaForCausalLM进行批量并行模型推理，通过torchrun等方式启动 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ interactive_generation.py | 基于collie.LlamaForCausalLM进行张量并行模型推理，通过torchrun等方式启动 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ pipeline_generation.py | 基于collie.LlamaForCausalLM进行流水线并行模型推理，通过torchrun等方式启动 |
|│&ensp;&ensp;&ensp;├─ monitor.py | 演示collie.utils.StepTimeMonitor（记录每个step的时间）的使用 |
|│&ensp;&ensp;&ensp;├─ **one_sentence_overfitting** |  |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ 3d_parallelism.py | 基于collie.LlamaForCausalLM，通过3D并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ 3d_parallelism_chatglm.py | 基于collie.ChatGLMForCausalLM，通过3D并行，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ inplace_tp.py | 基于collie.LlamaForCausalLM，通过InplaceSGD+张量并行，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ inplace_zero3_with_hf.py | 适配transformers.LlamaForCausalLM，通过InplaceSGD+数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ lora_zero3.py | 基于collie.LlamaForCausalLM，通过LoRA+数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ lora_zero3_moss.py | 基于collie.MossForCausalLM，通过LoRA+数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ peft_zero3_with_hf.py | 适配各种peft方式以及OPT-125M模型，通过数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;├─ zero3.py | 基于collie.MossForCausalLM，通过数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;│&ensp;&ensp;&ensp;└─ zero3_with_hf.py | 适配transformers.AutoModelForCausalLM，通过数据并行+ZeRO3，过拟合一句话 |
|│&ensp;&ensp;&ensp;└─ **servering** |  |
|│&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;└─ servering_with_training.py | 演示collie.utils.GradioProvider（用户输入文本，模型异步生成）的使用 |
|├─ requirements.txt | 描述CoLLiE安装主要依赖的包 |
|└─ setup.py | 安装CoLLiE |

&ensp; &ensp; 在接下来的Tutorial中，您将会看到

&emsp; （等最后确定了再填进去）